In [1]:
from dotenv import load_dotenv

load_dotenv('../../.env')

False

# Document Loaders

In [ ]:
from langchain.document_loaders import CSVLoader

# Load data from a CSV file using CSVLoader
loader = CSVLoader("./data/data.csv")
documents = loader.load()

# Access the content and metadata of each document
for document in documents:    
		content = document.page_content    
		metadata = document.metadata     

In [4]:
from langchain.document_loaders import WikipediaLoader

# Load content from Wikipedia using WikipediaLoader
loader = WikipediaLoader("Machine_learning")
document = loader.load()

# Models

In [5]:
from langchain import HuggingFaceHub

llm_mistral = HuggingFaceHub(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    model_kwargs={'temperature':0.5,"max_length": 64,"max_new_tokens":512}
)


In [10]:
from langchain_community.chat_models.huggingface import ChatHuggingFace


chat_mistral = ChatHuggingFace(llm=llm_mistral)


WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


In [12]:
from langchain_core.messages import HumanMessage, AIMessage
messages = [
    HumanMessage(
        content="Hi"
    ),
    AIMessage(
        content="I am a helpful assistant."
    ),
    HumanMessage(
        content="What is the capital of France?"
    ),
]


In [14]:
print(chat_mistral(messages))

content="<s>[INST] Hi [/INST]I am a helpful assistant.</s>[INST] What is the capital of France? [/INST] The capital city of France is Paris. Paris is one of the most famous cities in the world and is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is also home to many important cultural institutions and is a major global center for art, fashion, gastronomy, and culture. Paris is located in the north-central part of France and is the country's largest city. I hope that helps! Let me know if you have any other questions."


# Embedding Models

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs = {'device':'cpu'} )

In [17]:
# Embed a list of texts
embeddings = embeddings_model.embed_documents(
    ["Hi there!", "Oh, hello!", "What's your name?", "My friends call me World", "Hello World!"]
)
print("Number of documents embedded:", len(embeddings))
print("Dimension of each embedding:", len(embeddings[0]))

Number of documents embedded: 5
Dimension of each embedding: 384


# LLM Chain

In [18]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

template = """List all the colors in a rainbow"""
prompt = PromptTemplate(
    template=template, input_variables=[], output_parser=StrOutputParser()
)


In [20]:
llm_chain = LLMChain(prompt=prompt, llm=chat_mistral)

print(llm_chain.predict())

<s>[INST] List all the colors in a rainbow [/INST] A rainbow is an optical and meteorological phenomenon that is caused by reflection, refraction, and dispersion of light in water droplets. The colors of the rainbow are not actually present in the rainbow itself, but rather are an optical illusion created by the way light interacts with water droplets. The colors of the rainbow are always in the same order and are:

1. Red
2. Orange
3. Yellow
4. Green
5. Blue
6. Indigo
7. Violet

This order is often remembered by the acronym ROYGBIV. Each color has a specific wavelength in the visible light spectrum, with red having the longest wavelength and violet having the shortest.


In [22]:
prompt = PromptTemplate.from_template(
    "List all the colors in a {item}."
)
runnable = prompt | chat_mistral | StrOutputParser()
print(runnable.invoke({"item": "rainbow"}))

<s>[INST] List all the colors in a rainbow. [/INST] A rainbow is an optical and meteorological phenomenon that is caused by reflection, refraction and dispersion of light in water droplets. It appears as a multicolored circular arc. The colors in a rainbow, in order from the inner most band to the outermost band, are:

1. Red
2. Orange
3. Yellow
4. Green
5. Blue
6. Indigo
7. Violet

This acronym, "ROYGBIV," is often used to remember the order of the colors in a rainbow.


# Sequential

In [23]:
post_prompt = PromptTemplate.from_template(
    """You are a business owner. Given the theme of a post, write a social media post to share on my socials.

Theme: {theme}
Content: This is social media post based on the theme above:"""
)

review_prompt = PromptTemplate.from_template(
    """You are an expert social media manager. Given the presented social media post, it is your job to write a review for the post.

Social Media Post:
{post}
Review from a Social Media Expert:"""
)

In [24]:
chain = (
    {"post": post_prompt | llm_mistral | StrOutputParser()}
    | review_prompt
    | llm_mistral
    | StrOutputParser()
)

In [25]:
print(chain.invoke({"theme": "Having a black friday sale with 50% off on everything."}))

You are an expert social media manager. Given the presented social media post, it is your job to write a review for the post.

Social Media Post:
You are a business owner. Given the theme of a post, write a social media post to share on my socials.

Theme: Having a black friday sale with 50% off on everything.
Content: This is social media post based on the theme above:

📣🛍️ Attention all shoppers! 🛍️📣

It's that time of year again! Our Black Friday Sale is here and it's bigger and better than ever before! 🤩

🎁🛍️ For the next 24 hours, enjoy 50% off on everything in our store! That's right, HALF OFF! 😱

Whether you're looking for a little something for yourself or searching for the perfect holiday gift for a loved one, now is the time to shop! 🎁

🛍️🛍️ Don't miss out on this amazing deal! Head to our website or visit us in store to take advantage of these savings. 🛍️🛍️

Happy shopping, and we can't wait to help you find the perfect items for your cart! #BlackFriday #Sale #Shopping #Holi

# Memory